In [16]:
import torch
import random

from sent_order.conll import Corpus, Document

from torch.nn import functional as F
from itertools import islice
from tqdm import tqdm_notebook
from boltons.iterutils import pairwise

In [2]:
model = torch.load(
    '../../data/coref.4.bin',
    map_location={'cpu': 'cpu'},
)

In [3]:
model.eval()

Coref(
  (encode_doc): DocEncoder(
    (embeddings): WordEmbedding(39414, 300)
    (lstm): LSTM(300, 200, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.5)
  )
  (score_spans): SpanScorer(
    (attention): Scorer(
      (score): Sequential(
        (0): Linear(in_features=400, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
    (width_embeddings): DistanceEmbedding(9, 20)
    (sm): Scorer(
      (score): Sequential(
        (0): Linear(in_features=1120, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
  )
  (score_pairs): PairScorer(
    (dist_embeddings): DistanceEmbedding(9, 20)
    (sa): Scorer(
      (score): Sequential(
        (0): Linear(

In [4]:
c = Corpus.from_combined_file('../../data/test.conll')

In [11]:
def score(spans):
    return sum([sum(F.softmax(span.sij, 0)[:-1]) for span in spans])

In [18]:
correct, total = 0, 0
for doc in c.documents[:10]:
    for s1, s2 in pairwise(doc.sents()):

        d1 = Document(s1 + s2)
        d2 = Document(s2 + s1)

        try:

            print(model.predict(d1), '---', model.predict(d2))

            spans1 = model(d1)
            spans2 = model(d2)

            if score(spans1) > score(spans2):
                correct += 1

            total += 1

        except Exception as e:
            print(e)

[{(16, 16), (3, 3)}] --- []
[] --- []
[] --- [{(11, 11), (3, 6)}]
[] --- []
[] --- []
[] --- []
[] --- []
[] --- []
[] --- [{(0, 0), (25, 25)}]
[] --- [{(27, 28), (54, 54)}]
[{(40, 41), (27, 28)}] --- [{(3, 4), (38, 39)}]
[] --- []
[] --- []
[{(15, 16), (19, 20)}] --- [{(1, 2), (30, 31)}]
[] --- []
[] --- []
[{(9, 10), (26, 27)}] --- [{(7, 8), (35, 36)}]
[] --- []
[] --- []
[] --- []
[] --- []
[] --- []
[{(36, 36), (45, 45), (2, 2)}] --- [{(19, 19), (5, 5), (25, 25), (10, 10)}]
[{(5, 5), (19, 19), (10, 10)}] --- [{(12, 12), (7, 7), (21, 21)}]
[] --- []
[] --- []
[{(36, 36), (25, 25), (3, 3)}, {(21, 22), (32, 32)}] --- [{(3, 4), (14, 14)}, {(27, 27), (7, 7), (18, 18)}]
[{(3, 4), (14, 14)}, {(27, 27), (7, 7), (18, 18)}] --- [{(28, 28), (17, 17), (3, 3)}, {(13, 14), (24, 24)}]
[] --- []
[] --- []
[{(3, 4), (11, 11)}] --- [{(10, 11), (5, 5)}]
[] --- []
[] --- []
[{(5, 5), (17, 17)}, {(22, 23), (2, 3)}] --- [{(15, 15), (3, 3)}, {(20, 21), (0, 1)}]
[{(15, 15), (3, 3)}, {(20, 21), (0, 1), (46

In [10]:
correct / total

ZeroDivisionError: division by zero